In [709]:
from tabulate import tabulate
import pandas as pd
import pandas.io.common
import numpy as np
import os

In [710]:
characters=['\'',
            ',']

columns_to_delete=['energy','kwh']
rows_to_delete=['kwh','kWh']

In [711]:
def dfToLowerCase(df):
    df = df.apply(lambda x: x.astype(str).str.lower())
    return df

In [712]:
def removeUnwantedCharacter(df,characters):
    for c in characters:
            for column in df:
                if df[column].dtype == object:
                    df[column] = df[column].replace({c:''}, regex=True)
    return df

In [713]:
def searchForDate(df):
    print('searching for date')
    foundDate=''
    dates18=['2018','FY18','18/19']
    dates19=['2019','FY19','19/20']
    dates20=['2020','FY20','20/21']
    dates21=['2021','FY21','21/22']
    datesList=[dates18,dates19,dates20,dates21]
    print('collecting rows')
    try:
        rows=[df.loc[1].values.flatten().tolist(),
        df.loc[0].values.flatten().tolist()]
    except KeyError:#occurs when no second row for some columns
        rows=[df.loc[0].values.flatten().tolist()]
    #print(rows)
    for row in rows:
        #print('Searching for row: ',row)
        for item in row:
            #print('item: ',item)
            for dates in datesList:
                #print('searching for date',dates)
                if any(date in item for date in dates):
                    #print('found date: ',dates[0])
                    foundDate = dates[0]

    for dates in datesList:
        if any(date in df.columns for date in dates):
            foundDate = dates[0]
    return foundDate



In [714]:
def getdatafromrow(df):
    #print('getting data from row')
    #print(tabulate(df, headers='keys', tablefmt='psql'))
    df = df.reset_index(drop=True)
    #print(tabulate(df, headers='keys', tablefmt='psql'))
    values = df.loc[0].values.flatten().tolist()
    values.pop(0)
    data = ''
    #print(values)
    while data == '':
        #print('Checking items')
        for item in reversed(values):#could use reversed(values)

            if item !='nan':

                #print(item)
                numbers=[]
                numbers = [int(s) for s in item.split() if s.isdigit()]#extract integers from the string
                #print('Numbers: ',numbers)
                if numbers!=[]:
                    data = sorted(numbers, reverse=True)[0] #take the largest number found - to discount errors in the OCR process

                else:#if no integers are found, try and find floats
                    for token in item.split():
                        try:
                            #print (float(token))
                            numbers.append(float(token))
                        except ValueError:
                            pass
                    if numbers!=[]:
                        data = sorted(numbers, reverse=True)[0]
        break
    return data


In [715]:
number = '0.42'
for token in number.split():
    try:
        print (float(token))
    except ValueError:
        pass

0.42


In [716]:
a=[0,123]
x = sorted(a, reverse=True)
x

[123, 0]

In [717]:
def extractDataByPhrase(df,wantedPhrases,**kwargs):
    #print('extracting data by phrase')
    #print(tabulate(df, headers='keys', tablefmt='psql'))
    #print(wantedPhrases)
    unwantedPhrases = kwargs.get('unwantedPhrases', None)
    sumIfMultiple = kwargs.get('sumIfMultiple', True)
    returnData = ''
    '''
    Filters the Dataframe to df_f which only includes rows with the filter phrases defined & without unwanted phrases
    '''
    df_f=[]
    if len(wantedPhrases)==1:
        #print('Single wanted phrases detected')
        df_f = df[df[0].str.contains(wantedPhrases[0])]#filters to rows where the label contains the wanted phrase
    else:
        print('multiple wantedPhrases detected')
        print(wantedPhrases)
        #print('searching for: '+wantedPhrases[0])
        df_f = df[df[0].str.contains(wantedPhrases[0])]#search for first wanted phrase & create new dataframe
        #print('data found from first phrase: ')
        #print(tabulate(df_f, headers='keys', tablefmt='psql'))
        print(df[0])
        for i in range(1, len(wantedPhrases)):#search for the rest of the phrases & concat the dataframe
            print('Data faound from phrase '+str(i)+': '+wantedPhrases[i])
            print(df[0].str.contains(wantedPhrases[i]))
            print(tabulate(df[df[0].str.contains(wantedPhrases[i])], headers='keys', tablefmt='psql'))
            df_f=pd.concat([df_f,df[df[0].str.contains(wantedPhrases[i])]])
        df_f = df_f.drop_duplicates(keep='first')
        #print('Finished Searching for multiple phrases')
        #print(tabulate(df_f, headers='keys', tablefmt='psql'))
    print('unwanted phrases: ',unwantedPhrases)

    
    if unwantedPhrases is not None:
        print('dropping unwanted phrases')
        df_f = df_f.reset_index(drop=True)#resets the index
        for phrase in unwantedPhrases:
            df_f = df_f[~df_f[0].str.contains(phrase)]#'filters out the unwanted phrases
            
    df_f = df_f.reset_index(drop=True)#resets the index
    
    
    '''
    extracting data from the dfiltered dataframe
    if only 1 line, extracts that
    if multiple rows, extracts them all, and sums depending on argument given
    '''
    
    if len(df_f)==1:#if there is only 1 line of scope 1 data
        returnData = getdatafromrow(df_f)#extract the value from that row

    elif len(df_f)>1 and sumIfMultiple:#if multiple scope 1 values found

        #if there's a total scope 1 value, take that
        temp_df = df_f[df_f[0].str.contains("total")]
        if not temp_df.empty:
            returnData = getdatafromrow(temp_df)
            temp_df=[]

        #otherwise add all scope 1 values together
        else:
            values=[]#holds each scope 1 value
            for i in range(len(df_f)):#for each row
                #print('getting data for row: '+str(i))
                #print('Scope 1 Value: '+scope1_df.filter(items = [i], axis=0))
                values.append(getdatafromrow(df_f.filter(items = [i], axis=0)))
            values = list(filter(None, values))
            #print(values)
            returnData = sum(values)

    elif len(df_f)>1 and sumIfMultiple==False:
        #print('multiple phrases detected. No Sum argument given. giving first value in values columns.')
        returnData = getdatafromrow(df_f.head(1))
    
    return returnData
            
'''
HERE
'''    


'\nHERE\n'

In [718]:
import csv

def handleParseError(folder,file):
   with open(folder+'\\'+file, 'r') as read_obj:
      csv_reader = csv.reader(read_obj)
      list_of_csv = list(csv_reader)
      columnLen=len(list_of_csv[0])
      
      #print(columnLen)
      outputList=[]
      for row in list_of_csv:
         outputList.append(row[:columnLen])
      #print(outputList)
      df=pd.DataFrame(outputList)
      return df

In [719]:
def dropUnwantedRows(df,filterPhrases):
    print('Dropping Unwanted Rows')
    columnsToDelete=[]
    '''
    Can use to drop columns with phrases like KWH we dont want to accidently collect data for
    '''
   
    for phrase in filterPhrases:
        df[~df[0].str.contains(phrase)]
    
    return df


In [720]:
def dropUnwantedColumns(df,columns_to_delete):
    #print('Dropping Unwanted Columns')
    '''
    Import error where NaN values are sometimes imported as strings 'nan'
    Can mostly be ignored, but needs handling if first column are all NaN
    '''
    if all(i == 'nan' for i in df[0].values):
        df=df.drop(df.columns[0], axis=1)
        df.columns=range(df.shape[1])


    df = df.dropna(how='all', axis=1)#remove empty columns
    toDelete=[]
    columns = df.loc[:, df.columns != df.columns[0]].columns
    #print(tabulate(df, headers='keys', tablefmt='psql'))
    for column in columns:
        #print(column)
        if df[column].dtype == object:
            #print(str(df[column].iloc[0]))
            if any(word in str(df[column].iloc[0]) for word in columns_to_delete):
                toDelete.append(column)
            #print(str(df[column].iloc[1]))
            if len(df[column])>1 and any(word in str(df[column].iloc[1]) for word in columns_to_delete):
                if column not in toDelete:
                    toDelete.append(column)
    for column in toDelete:
            df = df.drop(column, 1)
    return df
    

In [723]:
from pandas.io.parsers import ParserError
folder = r'C:\Users\Clamfighter\Documents\GitHub\pdf-table-reader\Filtered\Filtered - Copy'

output=[]

for file in os.listdir(folder):
    try:
        if file.endswith(".csv"):
            print('file: '+file)
            try:
                df = pd.read_csv(folder+'\\'+file,encoding='iso-8859-1',header=None)
            except ParserError:
                df = handleParseError(folder,file)
            print('Original Dataframe ')
            print(tabulate(df, headers='keys', tablefmt='psql'))
            if len(df.columns)>1:
                df = df.dropna(how='all')#remove empty rows
                df = df.reset_index(drop=True)
                df = dfToLowerCase(df) #making df lower case
                df = removeUnwantedCharacter(df,characters)#remove unwanted characters from the whole of the dataframe  
                df = dropUnwantedColumns(df,columns_to_delete)#drop columns if their header is unwanted e.g units
                
                
                df = dropUnwantedRows(df,rows_to_delete)
                print('Altered Dataframe')
                print(tabulate(df, headers='keys', tablefmt='psql'))
                if len(df.columns)>1:
                    foundDate = searchForDate(df)
                    
                    scope1 = extractDataByPhrase(df,['scope 1',
                                                    'direct combustion',
                                                    'emissions from burning of gas',
                                                    'emissions from burning of fuel',
                                                    'emissions from combustion of gas',
                                                    'emissions from combustion of fuel'],
                                                
                                                unwantedPhrases=['scope 1\+2',
                                                    'scope 1 \+ 2',
                                                    'scope 1 \+2',
                                                    'scope 1\+ 2',
                                                    'kwh'])
                    scope2 = extractDataByPhrase(df,['scope 2',
                                                    'indirect',
                                                    'emissions from electricity',
                                                    'emissions from purchased electricity'],
                                                    
                                                    unwantedPhrases=['scope 1','kwh'])
                 
                    scope3 = extractDataByPhrase(df,['scope 3','travel'],
                                                    unwantedPhrases=['kwh'])
                    intensity = extractDataByPhrase(df,['intensity'])
                    totalemissions= extractDataByPhrase(df,['total gross co2e',
                                                        'total gross emissions',
                                                        'total net emissions',
                                                        'total emissions',
                                                        'total co2e',
                                                        'total gross co?e',
                                                        'total co?e',
                                                        'total gross coze',
                                                        'total coze',
                                                        'all scopes'],
                                                        unwantedPhrases=['kwh'],
                                                        sumIfMultiple=False)

                    print('Date: '+foundDate)
                    print('scope 1 value: ',scope1)
                    print('scope 2 value: ',scope2)
                    #print('scope 1 + 2 value: ',scope12)
                    print('scope 3 value: ',scope3)
                    print('Intensity: ', intensity)
                    print('total emissions: ',totalemissions)
                    output.append([file[:-4],foundDate,scope1,scope2,scope3,intensity,totalemissions])
            else:
                print('Not enough data found')
    except pandas.io.common.EmptyDataError:
        print('Dataframe Empty.')
        
import numpy as np
df_output = pd.DataFrame(output,columns=['Company Number','Date','Scope 1','Scope 2','Scope 3','Intensity','Total Emissions'])
df_output.to_csv(folder+'\\output.csv',index=False)
df_output

file: 00034871.csv
Original Dataframe 
+----+-----+
|    |   0 |
|----+-----|
|  0 | nan |
+----+-----+
Not enough data found
file: 00457936.csv
Original Dataframe 
+----+----------------------------------------------------+---------+-----------+
|    | 0                                                  | 1       | 2         |
|----+----------------------------------------------------+---------+-----------|
|  0 | nan                                                | Units   | 2020      |
|  1 | ENERGY INPUTS                                      | nan     | nan       |
|  2 | Total Purchased Electricity                        | kWh     | 964,862   |
|  3 | Total Purchased Gas                                | kWh     | 826,445   |
|  4 | Total Purchased Transport Fuels                    | kWh     | 97,112    |
|  5 | Total Energy Use                                   | kWh     | 1,888,419 |
|  6 | EMISSIONS                                          | nan     | nan       |
|  7 | Scope 1 

,Company Number,Date,Scope 1,Scope 2,Scope 3,Intensity,Total Emissions
0,00457936,2020,164121,224948,3455,,
1,00752082,2020,2961.49,1099.93,32074.7,0.47,
2,00895081,2020,3220,2346,108,75.65,
3,01083749,2019,1537.8,,,,
4,02343739,2019,,261315,,,
5,02686054,,,,,,
6,02831840,2020,3220,2346,108,75.65,
7,03185729,,943334,,,,
8,03203859,,31.8,142.18,353,,
9,04924233,2020,18964,18964,,,


In [722]:
'''
BUGLIST:
# searching for | breaks the search and returns all values which ultimatly returns no number values
example 199171


majority of time, newest data is on the left, so we collect that data.
sometimes newest data is on the right. we collect newest date, so sometimes data does not match date
example 4637868

'Emissions from the combustion of gas' not being found ion some csvs
example 9786835

'''

"\nBUGLIST:\n# searching for | breaks the search and returns all values which ultimatly returns no number values\nexample 199171\n\n\nmajority of time, newest data is on the left, so we collect that data.\nsometimes newest data is on the right. we collect newest date, so sometimes data does not match date\nexample 4637868\n\n'Emissions from the combustion of gas' not being found ion some csvs\nexample 9786835\n\n"

In [10]:
list = [1,2,3,4,5,6,7,8,9]
while list:
    print (list.pop(int(len(list)/2)))

5
6
4
7
3
8
2
9
1


In [1]:
import fitz
import cv2
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
filterPhrases = [
    'Scope 1',
    'Scope 2',
    'Scope 3',
    'scope 1',
    'scope 2',
    'scope 3',
    ]


    
def convert_pdf_to_image(fic):
    #open your file
    mat = fitz.Matrix(300 / 72, 300 / 72)  # sets zoom factor for 300 dpi
    pages = fitz.open(fic)
    for page in pages:
        pix = page.get_pixmap(matrix=mat)
        number=("page-%04i" % page.number)
        print(number)
        pix.pil_save(number+'.tiff', format="TIFF", dpi=(300,300))
        im = Image.open(number+'.tiff')
        im.save(number+'.jpeg')
        os.remove(number+'.tiff')
        image = cv2.imread(number+'.jpeg')
        text = str(pytesseract.image_to_string(image))
        if not any(word in text for word in filterPhrases):
            os.remove(number+'.jpeg')



convert_pdf_to_image(r'F:\Data Mining\CO2Extraction\00204368\00204368_fullPDF.pdf')

page-0000
page-0001
page-0002
page-0003
page-0004
page-0005
page-0006
page-0007
page-0008
page-0009
page-0010
page-0011
page-0012
page-0013
page-0014
page-0015
page-0016
page-0017
page-0018
page-0019
page-0020
page-0021
page-0022
page-0023
page-0024
page-0025
page-0026
page-0027
page-0028
page-0029
page-0030
page-0031
page-0032
page-0033
page-0034
page-0035
page-0036
page-0037
page-0038
page-0039
page-0040
page-0041
page-0042
page-0043
page-0044
page-0045
page-0046
page-0047
page-0048
page-0049
page-0050
page-0051
page-0052
page-0053
page-0054
page-0055
page-0056
page-0057
page-0058
page-0059
page-0060
page-0061
page-0062
page-0063
page-0064
page-0065
page-0066
page-0067
page-0068
page-0069
page-0070
page-0071
page-0072
page-0073
page-0074
page-0075
page-0076
page-0077
page-0078
page-0079
page-0080
page-0081
page-0082
page-0083
page-0084
page-0085
page-0086
page-0087
page-0088
page-0089
page-0090
page-0091
page-0092
page-0093
page-0094
page-0095
page-0096
page-0097
page-0098
page-0099


In [15]:
import fitz
import cv2
from PIL import Image
import pytesseract

companyNumber = '00204368'
folder='F:\Data Mining\CO2Extraction'
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
filterPhrases = [
    'Scope 1',
    'Scope 2',
    'Scope 3',
    'scope 1',
    'scope 2',
    'scope 3',
    ]


tempImagePath=(folder+'\\'+companyNumber+'\\images')
os.mkdir(tempImagePath)

imageCount = 0
dataFound=False
bufferPageCount=0
    

#open your file
mat = fitz.Matrix(300 / 72, 300 / 72)  # sets zoom factor for 300 dpi
pages = fitz.open(folder+'\\'+companyNumber+'\\'+companyNumber+'_fullPDF.pdf')
print(pages[0])
print('length ',len(pages))
pageNumbers = list(range(0, len(pages)))

while pageNumbers:
    pageNumber =pageNumbers.pop(int(len(pageNumbers)/2))
    print(pageNumber)
    page = pages[pageNumber]
    pix = page.get_pixmap(matrix=mat)

    number=("page-%04i" % page.number)
    

    print(number)

    pix.pil_save(tempImagePath+'\\'+number+'.tiff', format="TIFF", dpi=(300,300))
    im = Image.open(tempImagePath+'\\'+number+'.tiff')
    im.save(tempImagePath+'\\'+number+'.jpeg')
    os.remove(tempImagePath+'\\'+number+'.tiff')
    image = cv2.imread(tempImagePath+'\\'+number+'.jpeg')
    text = str(pytesseract.image_to_string(image))
    if not any(word in text for word in filterPhrases):
        os.remove(tempImagePath+'\\'+number+'.jpeg')
    else:
        dataFound=True
        print('Data Found')
    if dataFound==True:
            bufferPageCount+=1
            print('Buffer Page Check: '+str(bufferPageCount))
            if bufferPageCount==5:
                print('Buffer page count reached')
                pageNumbers=[]
                print(pageNumbers)





page 0 of F:\Data Mining\CO2Extraction\00204368\00204368_fullPDF.pdf
length  112
56
page-0056
55
page-0055
57
page-0057
54
page-0054
58
page-0058
53
page-0053
59
page-0059
52
page-0052
60
page-0060
51
page-0051
61
page-0061
50
page-0050
62
page-0062
49
page-0049
Data Found
Buffer Page Check: 1
63
page-0063
Buffer Page Check: 2
48
page-0048
Buffer Page Check: 3
64
page-0064
Buffer Page Check: 4
47
page-0047
Buffer Page Check: 5
Buffer page count reached
[]


In [30]:
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

im = Image.open('page-0049.tiff')
im.save('page0049.jpeg')
image = cv2.imread('page0049.jpeg')
text = str(pytesseract.image_to_string(image))
print(text)

! Finsbury Food Group

|
| Annual Report and Accounts 2021

 

Streamlined Energy and Carbon Reporting

The UK Government's Streamlined Energy and Carbon Reporting (SECR) policy was implemented on 1 April 2019. The table below represents Finsbury
Food Group’s energy use and associated greenhouse gas (GHG) emissions from electricity and fuel in the UK for the year ended 26 June 2021. The data
covers seven manufacturing sites in the UK. .

UK Greenhouse Gas Emissions and Energy Use Data

52 weeks ending 52 weeks ending

26 June 2021 27 june 2020
Energy consumption used to calculate emissions (kWh) kWh kWh
Total Energy Consumption (kWh) 102,577,469 106,904,756
Energy consumption break down (kWh):
Natural gas 69,487,690 67,208,470
Electricity 32,624,756 38,714,433
Transport 162,423 433,331
Diesel 149,094 367,909
LPG 153,506 180,613

TonnesCO,e Tonnes CO,e

Scope 1 emissions in metric tonnes CO,e

Natural gas 12,747.99 12,357.62
Refrigerant emissions ; 497.37 179.90
Diesel 37.69 93.00
LPG 3